In [ ]:
import numpy as np
import pandas as pd
from math import radians, cos, sin, sqrt, atan2

# 거리 계산 함수 (Haversine 공식)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # 지구의 평균 반지름 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2) * sin(dlat/2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2) * sin(dlon/2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# 데이터 읽기
df_districts = pd.read_excel('달서구 행정구역 중심좌표.xlsx')
df_hospitals = pd.read_csv('달서구 인프라 좌표.csv',encoding = 'cp949')

# '위도'와 '경도'를 str로 변환 후, 숫자 추출
df_hospitals['위도'] = df_hospitals['위도'].astype(str).str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)
df_hospitals['경도'] = df_hospitals['경도'].astype(str).str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)

# 인프라 점수 계산 함수
def calculate_infrastructure_score(lat, lon, hospitals, radius=0.5):
    scores = []
    for _, row in hospitals.iterrows():
        dist = haversine(lat, lon, row['위도'], row['경도'])
        if dist <= radius:
            score = 1 / (dist + 0.1)  # 역수 방식의 점수, 0으로 값이 나오는 것을 방지하기 위해 0.1 추가
            scores.append(score)
    return sum(scores)  # 또는 np.mean(scores)로 평균 점수

# 각 행정동에 대해 인프라 점수 계산
df_districts['인프라점수'] = df_districts.apply(
    lambda row: calculate_infrastructure_score(row['위도'], row['경도'], df_hospitals),
    axis=1
)

print(df_districts[['행정구역', '인프라점수']])


   행정구역       인프라점수
0   성당동   84.628450
1   두류동   37.224575
2   본리동  136.014102
3   감삼동  138.261700
4   죽전동   22.015173
5   장기동   84.658347
6   용산동  171.164155
7   이곡동  145.972079
8   신당동   59.606841
9   월성동   87.179382
10  진천동  242.826574
11  유천동   55.529302
12  상인동  147.305652
13  도원동  167.723794
14  송현동   91.305455
15   본동   51.825570


In [ ]:
df_districts.to_excel('달서구 행정구역 인프라지수.xlsx', index=False)